In [1]:
import numpy as np
import warnings


warnings.filterwarnings("ignore")

In [2]:
def getSpectrum(getA, a, b, N):
    A = getA(a, b, N)
    X1 = np.arange(N)
    lambda_, lambda_0 = 1, 0
    while abs(lambda_ - lambda_0) > 0.01:
        X = X1
        X1 = A.dot(X)
    
        lambda_0 = lambda_
        lambda_ = np.sum(X1 * X1) / np.sum(X1 * X)
        
    return lambda_

In [3]:
def RicherdsonExtrapolation(func, a, b, EPS=10, p=2, q=1, r=2, N=2, S=25):
    U = np.zeros((S,S))
    R = np.zeros((S,S))
    p_eff = np.zeros((S,S))

    s = 1
    U[0][0] = getSpectrum(func, a, b, N)
    while True:
        
        U[s][0] = getSpectrum(func, a, b, r**s*N)
        for n in range(s):
            
            R[s][n] = (U[s][n] - U[s-1][n]) / (r**(p + n*q) - 1)
            p_eff[s][n] = np.log(abs(R[s-1][n] / R[s][n])) / np.log(r)
            U[s][n + 1] = U[s][n] + R[s][n]
            
            if 100 * abs(R[s][n] / U[s][n]) < EPS:
                pt = p*(s-1)+n*q | p
                return (U[s][n], R[s][n], p_eff[s][n-1], pt, s, n, U, R, p_eff) 
            
        s += 1
        if s > 23: 
            print('ololo')
            return (U[s-1][s-1], R[s-1][s-1], p_eff[s-1][s-3], p + s*q, s, s, U, R, p_eff)

In [4]:
def PrintTriangular(mas, i, lines=None):
    if lines is None:
        lines = len(mas)
        
    for line in range(1+lines):
        for n in range(line + 1 - i):
            print('{0:7.4f}'.format(mas[line][n]), end=' ')
        print()

In [5]:
def show_answer(ans):
    re = " \
Ответ: {0:12.10f}\n \
Погрешность: {1:12.10f}\n \
Ошибка: {2:6.4f}%\n \
Эффективный порядок точности: {3:7.10f}\n \
Теоретический порядок точности: {4:7.5f}\n \
    ".format(ans[0], ans[1], 100*np.abs(ans[1]/ans[0]), ans[2], ans[3])
    print(re)
    PrintTriangular(ans[-3], 0, ans[4])
    PrintTriangular(ans[-2], 1, ans[4])
    PrintTriangular(ans[-1], 2, ans[4])

#### Поиск минимального собсвенного значения задачи Штурма-Лиувилля при помощи алгоритма обратной итерации с использованием методики Ричердсона.
$$
\begin{equation*}
 \begin{cases}
   U_{xx} + aU_x + \lambda U = 0, 
   \\
   U(0) = 0, U(1) = 0, 
   \\
   x \in [0, 1]
 \end{cases}
\end{equation*}
$$

1. Данной задаче будет соответсвовать матрица A в бесконечномерном пространсве
2. Если использовать в качестве производных их конечно разностную апроксимацию со вторым порядком точности, то коэффициенты данной матрицы будут выражаеться по формулам:
   - $a_{n,n-1} = \frac{1}{h^2} - \frac{a}{2h}$, $n = 2,..,N$
   - $a_{n,n} = -\frac{2}{h^2}$, $n = 1,..,N$
   - $a_{n,n+1} = \frac{1}{h^2} + \frac{a}{2h}$, $n = 1,..,N$
   
где a - любое число, h - шаг равномерной сетки по х

3. Таким образом, получим систему уравнений $Ay = \lambda y$, для которой можно применить метод обратной итерации, тогда 
   - $ \frac{(x^{(s+1)},x^{(s+1)})}{(x^{(s)},x^{(s+1)})} = \lambda_n \rightarrow  \lambda_{min}$, при $s \rightarrow +\infty$

## Задача 1: Найти минимальное с.з задачи   ШТ с точностью не хуже 3%
$$
\begin{equation*}
 \begin{cases}
   U_{xx} + 4U_x + \lambda U = 0, 
   \\
   U(0) = 0, U(1) = 0, 
   \\
   x \in [0, 1]
 \end{cases}
\end{equation*}
$$

In [6]:
def task1(a, b, N):
    h = (b - a) / N
    B = np.zeros((N+1, N+2))
    
    xn = a
    for n in range(1, N+1):
        xn += h 
        B[n][n-1] = 1/h**2 - 2/h
        B[n][n] = -2/h**2
        B[n][n+1] = 1/h**2 + 2/h

    A = np.linalg.inv(B[1:, 1:-1])     
    return A

In [7]:
#EPS - в процентах 
ans = RicherdsonExtrapolation(task1, a=0, b=1, EPS=3)
show_answer(ans)

 Ответ: -0.0860821943
 Погрешность: 0.0009279250
 Ошибка: 1.0780%
 Эффективный порядок точности: 1.4963242755
 Теоретический порядок точности: 3.00000
     
-0.1548 
-0.1081 -0.0926 
-0.0916 -0.0861 -0.0852 

 0.0155 
 0.0055  0.0009 


 1.4963 


## Задача 2: Найти минимальное с.з задачи ШТ с точностью не хуже 1%
$$
\begin{equation*}
 \begin{cases}
   U_{xx} + 9xU_x + \lambda U = 0, 
   \\
   U(0) = 0, U(1) = 0, 
   \\
   x \in [0, 1]
 \end{cases}
\end{equation*}
$$

Тогда матрица задачи будет задоваться:
   - $a_{n,n-1} = \frac{1}{h^2} - \frac{9x_n}{2h}$, $n = \overline{2, N}$
   - $a_{n,n} = -\frac{2}{h^2}$, $n = \overline{1, N}$
   - $a_{n,n+1} = \frac{1}{h:2} + \frac{9x_n}{2h}$, $n = \overline{1, N}$
   
где $x_n = x_0 + hn$

In [8]:
def task2(a, b, N):
    h = (b - a) / N
    B = np.zeros((N+1, N+2))
    
    xn = a
    for n in range(1, N+1):
        B[n][n-1] = 1/h**2 - 9*xn/(2*h)
        B[n][n] = -2/h**2
        B[n][n+1] = 1/h**2 + 9*xn/(2*h)
        xn += h 
        
    A = np.linalg.inv(B[1:, 1:-1])    
    return A

In [9]:
#EPS - в процентах 
ans2 = RicherdsonExtrapolation(task2, a=0, b=1, EPS=1)
show_answer(ans2)

 Ответ: -0.0573647375
 Погрешность: 0.0000936658
 Ошибка: 0.1633%
 Эффективный порядок точности: 1.8945604579
 Теоретический порядок точности: 7.00000
     
-0.1600 
-0.0926 -0.0702 
-0.0667 -0.0580 -0.0563 
-0.0597 -0.0574 -0.0573  0.0000 

 0.0225 
 0.0086  0.0017 
 0.0023  0.0001  0.0000 


 1.3768 
 1.8946  4.2099 


## Найти первое с.з. задачи: $ u(x) = \lambda \int_{-1}^{1} (x + \xi)^2u(\xi) \,d\xi$

1. Представим задачу в виде суммы используя формулу трапеций: 
   - $U_m = \lambda \sum_{n=1}^{N} \frac{(x_m + \xi_{n-1})^2U_{n-1} + (x_m + \xi_n)^2U_n}{2}h_n\ , m = \overline{1, N}$
2. Что соответсвует матричному уравнению $\overline U = \lambda A \overline U$, где коэффициенты матрицы выражаються как 
   - $\{a_{i,j}\} = (x_i + \xi_j)^2h$, если $j = 0; N$ 
   - $\{a_{i,j}\} = 2(x_i + \xi_j)^2h$, если $0 < j < N$ 
   - где $i, j = \overline{0, N}$
3. Сделаем замену $\nu = \frac{1}{\lambda}$, тогда получим уравнение $A \overline U = \nu \overline U$.
4. Таким образом задаче о поиске первого (наименьшего) с.з. $\lambda$, будет эквивалентна задача о поиске последнего (наибольшего) $\nu$. 
5. Поэтому будем использовать метод прямой итерации

In [10]:
def task3(a, b, N):
    h = (b - a) / N # по x будем брать такую же область
    A = np.zeros((N, N))
    
    x = a
    xi = a
    for i in range(N):
        xi = a
        A[i][0] = (x + xi)**2*h
        for j in range(1, N-1):
            xi += h
            A[i][j] = 2*(x + xi)**2*h
        
        xi += h
        A[i][N-1] = (x + xi)**2*h
        x += h
    
    return A

In [11]:
#EPS - в процентах 
ans3 = RicherdsonExtrapolation(task3, a=-1, b=1, EPS=1)
show_answer(ans3)

 Ответ: 2.9246389272
 Погрешность: -0.0147962711
 Ошибка: 0.5059%
 Эффективный порядок точности: 1.5745396035
 Теоретический порядок точности: 3.00000
     
 4.2361 
 3.3302  3.0282 
 3.0260  2.9246  2.9098 

-0.3020 
-0.1014 -0.0148 


 1.5745 


Сделаем обратную замену и получим первое с.з:

In [12]:
re = " \
Ответ: {0:12.10f}\n \
Погрешность: {1:12.10f}\n \
    ".format(1/ans3[0], abs(ans3[1])**(0.5)/ans3[0])
print(re)

 Ответ: 0.3419225501
 Погрешность: 0.0415914330
     


## Найти первое с.з. задачи: $ u(x) = \lambda \int_{0}^{\pi} sin(x + \xi)u(\xi) \,d\xi$

1. Построим совершенно аналагичные рассуждение предидущей задачи

In [13]:
def task4(a, b, N):
    h = (b - a) / N # по x будем брать такую же область
    A = np.zeros((N, N))
    
    x = a
    xi = a
    for i in range(N):
        xi = a
        A[i][0] = np.sin(x + xi)*h
        for j in range(1, N-1):
            xi += h
            A[i][j] = 2*np.sin(x + xi)*h
        
        xi += h
        A[i][N-1] = np.sin(x + xi)*h
        x += h
    
    return A

In [14]:
#EPS - в процентах 
ans4 = RicherdsonExtrapolation(task3, a=0, b=np.pi, EPS=1)
show_answer(ans4)

 Ответ: 79.3771239349
 Погрешность: 0.6359930603
 Ошибка: 0.8012%
 Эффективный порядок точности: 0.8539924425
 Теоретический порядок точности: 6.00000
     
16.4181 
39.7752 47.5608 
60.2333 67.0527 69.8372 
73.4358 77.8366 79.3771 80.0131 

 7.7857 
 6.8194  2.7845 
 4.4008  1.5406  0.6360 


 0.1912 
 0.6319  0.8540 


In [15]:
re = " \
Ответ: {0:12.10f}\n \
Погрешность: {1:12.10f}\n \
    ".format(1/ans4[0], abs(ans4[1])**(0.5)/ans4[0])
print(re)

 Ответ: 0.0125980881
 Погрешность: 0.0100468711
     
